# Grupo 6
## Integrantes 
- Pablo Sarzosa
- Elisa Herrera
- Doménica Rodríguez 
- Andrés Albarracín

# 8.5. Implementación de Redes Neuronales Recurrentes desde Cero

IEn esta sección implementaremos un RNN desde cero para un modelo de lenguaje a nivel de caracteres, de acuerdo con nuestras descripciones en la Sección 8.4. Tal modelo será entrenado en La máquina del tiempo de H. G. Wells. Como antes, comenzamos leyendo primero el conjunto de datos, que se presenta en la Sección 8.3.

In [6]:
use strict;
use warnings;
use Data::Dump qw(dump);
use AI::MXNet qw(mx);
use d2l;
use d2l::Vocab;
use d2l::SeqDataLoader;
IPerl->load_plugin('Chart::Plotly'); # Jupyter
#import Chart::Plotly 'show_plot'; # localmente

In [16]:
my ($batch_size, $num_steps) = (32, 35);
my ($train_iter, $vocab) = d2l->load_data_time_machine(batch_size      => $batch_size, 
                                                       num_steps       => $num_steps,
                                                       use_random_iter => 1);

CODE(0xc115110)Vocab=HASH(0xc0d0628)

In [17]:
print "vocab:", dump $vocab;

vocab:bless({
  _token_freqs => [
                    [" ", 32926],
                    ["e", 19781],
                    ["t", 15155],
                    ["a", 12752],
                    ["i", 11312],
                    ["o", 11157],
                    ["n", 11012],
                    ["s", 9313],
                    ["r", 8895],
                    ["h", 8841],
                    ["d", 6868],
                    ["l", 6658],
                    ["m", 4420],
                    ["u", 4328],
                    ["c", 4049],
                    ["f", 3749],
                    ["w", 3524],
                    ["g", 3512],
                    ["y", 3024],
                    ["p", 2867],
                    ["b", 2171],
                    ["v", 1435],
                    ["k", 1231],
                    ["x", 282],
                    ["j", 187],
                    ["z", 108],
                    ["q", 106],
                  ],
  idx_to_token => [
                    "<unk>",
  

1

In [18]:
# Iterator
while ( my $minibatch = $train_iter->() ) {
  print "\n\nX:", dump($minibatch->{data}->asarray), "\n";
  print "\n\nY:", dump($minibatch->{label}->asarray), "\n\n";
  last;
}



X:[
  [
    5 .. 8,
    1,
    6,
    16,
    1,
    10,
    5,
    8,
    1,
    16,
    6,
    14,
    9,
    1,
    11,
    5,
    13,
    2,
    7,
    8,
    5,
    6,
    7,
    2,
    11,
    1,
    21,
    2,
    5,
    7,
    18,
    1,
  ],
  [
    14,
    1,
    4,
    9,
    2,
    1,
    7,
    6,
    3,
    1,
    12,
    6,
    15,
    4,
    3,
    2,
    11,
    1,
    5,
    7,
    1,
    3,
    10,
    2,
    1,
    14,
    7,
    5,
    3,
    2,
    11,
    1,
    8,
    3,
    4,
  ],
  [
    5,
    1,
    7,
    2,
    22,
    2,
    9,
    1,
    3,
    4,
    12,
    23,
    2,
    11,
    1,
    6,
    16,
    1,
    5,
    3,
    1,
    14,
    7,
    3,
    5,
    12,
    2,
    24,
    20,
    2,
    9,
    5,
    13,
    2,
    7,
  ],
  [
    3,
    2,
    8,
    1,
    19,
    6,
    14,
    17,
    5,
    12,
    12,
    1,
    10,
    4,
    22,
    2,
    1,
    3,
    6,
    1,
    15,
    10,
    2,
    15,
    23,
    1,
    3,
    10,
    2,
   


## 8.5.1. Codificación One-Hot
Recuerde que cada token se representa como un índice numérico en train_iter. Alimentar estos índices directamente a una red neuronal puede dificultar el aprendizaje. A menudo representamos cada token como un vector de características más expresivo. La representación más fácil se llama codificación one-hot, que se presenta en la Sección 3.4.1.

En pocas palabras, asignamos cada índice a un vector unitario diferente: supongamos que el número de tokens diferentes en el vocabulario es N
  (len(vocab)) y los índices de los tokens van de 0 a N−1
 . Si el índice de un token es el número entero i
 , luego creamos un vector de todos los 0 con una longitud de N
  y coloque el elemento en la posición i
  a 1. Este vector es el vector caliente del token original. Los vectores one-hot con índices 0 y 2 se muestran a continuación.



In [19]:
print mx->nd->one_hot(mx->nd->array([0,2]),$vocab->len)->aspdl;


[
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
]


1

La forma del mini lote que muestreamos cada vez es (tamaño del lote, número de pasos de tiempo). La función one_hot transforma un minilote de este tipo en un tensor tridimensional con la última dimensión igual al tamaño del vocabulario (len(vocab)). A menudo transponemos la entrada para obtener una salida de forma (número de pasos de tiempo, tamaño del lote, tamaño del vocabulario). Esto nos permitirá recorrer de manera más conveniente la dimensión más externa para actualizar los estados ocultos de un minilote, paso a paso.

In [20]:
my $X=mx->nd->array([(0..9)])->reshape([2,5]);
$X = mx->nd->one_hot($X->transpose,28);

<AI::MXNet::NDArray 5x2x28 @cpu(0)>

## 8.5.2. Inicialización de los parámetros del modelo
A continuación, inicializamos los parámetros del modelo para el modelo RNN. El número de unidades ocultas num_hiddens es un hiperparámetro ajustable. Al entrenar modelos de lenguaje, las entradas y salidas son del mismo vocabulario. Por lo tanto, tienen la misma dimensión, que es igual al tamaño del vocabulario.


In [21]:
sub normal{
    my (@shape, $device)=@_;
    return mx->nd->random->normal(scale=>0.01, shape=>@shape, ctx=>$device);
}

In [22]:
sub get_params{
    my ($vocab_size,$num_hiddens,$device)=@_;
    my $num_inputs = $vocab_size;
    my $num_outputs = $num_inputs;
    # Hidden Layer parameters
    my $W_xh = normal([$num_inputs, $num_hiddens]);
    my $W_hh = normal([$num_hiddens, $num_hiddens]);
    my $b_h = mx->nd->zeros([$num_hiddens], ctx => d2l->try_gpu());
    # Output layer parameters
    my $W_hq = normal([$num_hiddens, $num_outputs]);
    my $b_q = mx->nd->zeros([$num_outputs], ctx => d2l->try_gpu());
    # Attach gradients
    #my $params = mx->nd->array(($W_xh, $W_hh, $b_h, $W_hq, $b_q));
    my $params = [$W_xh, $W_hh, $b_h, $W_hq, $b_q];
    for my $param (@{$params}){
        $param->attach_grad();
    }
    return $params;
}

8.5.3. Modelo RNN
Para definir un modelo RNN, primero necesitamos una función init_rnn_state para devolver el estado oculto en la inicialización. Devuelve un tensor lleno de 0 y con forma de (tamaño de lote, número de unidades ocultas). El uso de tuplas facilita el manejo de situaciones en las que el estado oculto contiene múltiples variables, que encontraremos en secciones posteriores.


In [23]:
sub init_rnn_state{
    my ($batch_size, $num_hiddens, $device)=@_;
    return mx->nd->zeros([$batch_size,$num_hiddens],ctx=>$device);
}

La siguiente función rnn define cómo calcular el estado oculto y la salida en un paso de tiempo. Tenga en cuenta que el modelo RNN recorre la dimensión más externa de las entradas para que actualice los estados ocultos H de un minilote, paso a paso. Además, la función de activación aquí usa el tanh
  función. Como se describe en la Sección 4.1, el valor medio de la tanh
  función es 0, cuando los elementos se distribuyen uniformemente sobre los números reales.


In [24]:
sub rnn{
    my ($inputs, $state, $params)=@_;
    my ($W_xh, $W_hh, $b_h, $W_hq, $b_q) = $params;
    my $H = $state;
    my $outputs = mx->nd->array();
    
    for my $X (@{$inputs}){
        $H = mx->nd->tanh()
    }
}

In [25]:
sub rnn {
my ($inputs, $state, $params) = @_;
# Shape of inputs: (num_steps, batch_size, vocab_size)
my ($W_xh, $W_hh, $b_h, $W_hq, $b_q) = @{$params};
my ($H) = @$state;
my @outputs = ();
# Shape of X: (batch_size, vocab_size)
for my $X (@{$inputs}) {
    $H = mx->nd->tanh(mx->nd->dot($X, $W_xh) + mx->nd->dot($H, $W_hh) + $b_h);
    my $Y = mx->nd->dot($H, $W_hq) + $b_q;
    #print @outputs,": ";
    push @outputs, $Y;
}
    #print @outputs->len;
    return (@outputs, $H);
}

Warning: Subroutine rnn redefined at reply input line 1.


Con todas las funciones necesarias definidas, a continuación creamos una clase para envolver estas funciones y almacenar parámetros para un modelo RNN implementado desde cero.

In [26]:
package RNNModelScratch;

sub new {
    my ($class, $vocab_size, $num_hiddens, $device) = @_;
    my $self = bless {}, $class;
    $self->{vocab_size} = $vocab_size;
    $self->{num_hiddens} = $num_hiddens;
    #my @params = get_params($vocab_size, $num_hiddens, $device);
    $self->{params} = get_params($vocab_size, $num_hiddens, $device);;
    $self->{init_state};
    $self->{forward_fn};
    return $self;
}

sub call {
    my ($self, $X, $state) = @_;
    $X = mx->nd->one_hot($X->transpose, $self->{vocab_size});
    return rnn($X, $state, $self->{params});
}

sub begin_state {
    my ($self, $batch_size, $ctx) = @_;
    return init_rnn_state($batch_size, $self->{num_hiddens}, $ctx);
}

1;

1

Warning: Useless use of hash element in void context at reply input line 10.

Useless use of hash element in void context at reply input line 11.

Subroutine new redefined at reply input line 3.

Subroutine call redefined at reply input line 15.

Subroutine begin_state redefined at reply input line 21.


Verifiquemos si las salidas tienen las formas correctas, por ejemplo, para garantizar que la dimensionalidad del estado oculto permanezca sin cambios.


In [36]:
my $num_hiddens = 512;
my $net = RNNModelScratch->new($vocab->len, $num_hiddens, d2l->try_gpu());
my $state = $net->begin_state($X->shape()->[0], d2l->try_gpu());
my $datos = $net->{params};
my ($Y) = $net->call($X->as_in_context(d2l->try_gpu()), $state);
my $new_state=$Y[28];

<AI::MXNet::NDArray 2x5x512 @cpu(0)>

In [37]:
print "Y shape: " , (dump $Y->shape) , "\n";
print "New state length: " , scalar(@{$new_state}) , "\n";
print "New state shape: " , (dump $new_state->[0]->shape) , "\n";

Y shape: [2, 5, 28]
New state length: 2
New state shape: [5, 512]


1

Podemos ver que la forma de salida es (número de pasos de tiempo ×
  tamaño del lote, tamaño del vocabulario), mientras que la forma del estado oculto sigue siendo la misma, es decir, (tamaño del lote, número de unidades ocultas).

## 8.5.4. Predicción
Primero definamos la función de predicción para generar nuevos caracteres siguiendo el prefijo proporcionado por el usuario, que es una cadena que contiene varios caracteres. Al recorrer estos caracteres iniciales en el prefijo, seguimos pasando el estado oculto al siguiente paso de tiempo sin generar ningún resultado. Esto se denomina período de calentamiento, durante el cual el modelo se actualiza (por ejemplo, actualiza el estado oculto) pero no hace predicciones. Después del período de calentamiento, el estado oculto es generalmente mejor que su valor inicializado al principio. Entonces generamos los caracteres predichos y los emitimos.


In [153]:
use AI::MXNet qw(mx); 

sub predict_ch8 {
    my ($prefix_, $num_preds, $net, $vocab, $device) = @_;
    
    my @prefix=split("",$prefix_);
    my $state = $net->begin_state(1,$device);
    my @outputs = ($vocab->{@prefix[0]});
    my $get_input = sub { mx->nd->array([$outputs[-1]], ctx => $device)->reshape([1,1]) };
    
    # Warm-up period
    foreach my $y (@$prefix[1..$#$prefix]) {
        my ($_, $state) = $net->($get_input->(), $state);
        push @outputs, $vocab->{$y};
    }
    
    # Predict `num_preds` steps
    for (1..$num_preds) {
        my ($y, $state) = $net->call($get_input->(), $state);
        push @outputs, int($y->argmax()->aspdl()->at(0));
    }
    return join('', map { $vocab->{idx_to_token}[$_] } @outputs);
}

Warning: Subroutine predict_ch8 redefined at reply input line 3.

Scalar value @prefix[0] better written as $prefix[0] at reply input line 8.


Ahora podemos probar la función predict_ch8. Especificamos el prefijo como viajero en el tiempo y hacemos que genere 10 caracteres adicionales. Dado que no hemos entrenado la red, generará predicciones sin sentido.


In [155]:
predict_ch8('time traveller ', 10, $net, $vocab, d2l->try_gpu());

<unk>oenxaxaxax

Warning: Use of uninitialized value $_ in array element at reply input line 22.


## 8.5.5. Recorte de degradado
Para una secuencia de longitud T
 , calculamos los gradientes sobre estos T
  pasos de tiempo en una iteración, lo que da como resultado una cadena de productos de matriz con longitud O (T)
  durante la retropropagación. Como se mencionó en la Sección 4.8, podría resultar en inestabilidad numérica, por ejemplo, los gradientes pueden explotar o desaparecer, cuando T
  es largo. Por lo tanto, los modelos RNN a menudo necesitan ayuda adicional para estabilizar el entrenamiento.

En términos generales, al resolver un problema de optimización, tomamos medidas de actualización para el parámetro del modelo, digamos en la forma vectorial x
 , en la dirección del gradiente negativo g
  en un mini lote. Por ejemplo, con η>0
  como la tasa de aprendizaje, en una iteración actualizamos x
  como x−ηg
 . Supongamos además que la función objetivo f
  se comporta bien, digamos, Lipschitz continua con constante L
 . Es decir, para cualquier x
  y yo
  tenemos

(8.5.1)
|f(x)−f(y)|≤L∥x−y∥.
 
En este caso podemos asumir con seguridad que si actualizamos el vector de parámetros por ηg
 , entonces

(8.5.2)
|f(x)−f(x−ηg)|≤Lη∥g∥,
 
lo que significa que no observaremos un cambio por más de Lη∥g∥
 . Esto es tanto una maldición como una bendición. Por el lado de la maldición, limita la velocidad de progreso; mientras que en el lado de la bendición, limita la medida en que las cosas pueden salir mal si nos movemos en la dirección equivocada.

A veces, los gradientes pueden ser bastante grandes y es posible que el algoritmo de optimización no converja. Podríamos abordar esto reduciendo la tasa de aprendizaje η
 . Pero, ¿y si rara vez obtenemos grandes gradientes? En este caso, tal enfoque puede parecer totalmente injustificado. Una alternativa popular es recortar el gradiente g
  proyectándolos de nuevo a una bola de un radio dado, digamos θ
  a través de

(8.5.3)
g←min(1,θ∥g∥)g.
 
Al hacerlo, sabemos que la norma del gradiente nunca excede θ
  y que el gradiente actualizado está completamente alineado con la dirección original de g
 . También tiene el efecto secundario deseable de limitar la influencia que cualquier minilote dado (y dentro de él, cualquier muestra dada) puede ejercer sobre el vector de parámetros. Esto otorga un cierto grado de robustez al modelo. El recorte de degradado proporciona una solución rápida a la explosión del degradado. Si bien no resuelve completamente el problema, es una de las muchas técnicas para aliviarlo.

A continuación, definimos una función para recortar los gradientes de un modelo que se implementa desde cero o un modelo construido por las API de alto nivel. También tenga en cuenta que calculamos la norma de gradiente sobre todos los parámetros del modelo.


In [156]:
sub grad_clipping {
    my ($net, $theta) = @_;
    
    if ($net->isa('AI::MXNet::Gluon::Block')) {
        my @params = map { $_->data } $net->collect_params->values;
    } else {
        my @params = $net->params;
    }
    
    my $norm = sqrt(sum(map { sum($_->grad ** 2)->asscalar } @params));
    if ($norm > $theta) {
        for my $param (@params) {
            $param->grad *= $theta / $norm;
        }
    }
}

Warning: Subroutine grad_clipping redefined at reply input line 1.


## 8.5.6. Capacitación
Antes de entrenar el modelo, definamos una función para entrenar el modelo en una época. Difiere de cómo entrenamos el modelo de la Sección 3.6 en tres lugares:

Diferentes métodos de muestreo para datos secuenciales (muestreo aleatorio y partición secuencial) darán como resultado diferencias en la inicialización de estados ocultos.

Recortamos los gradientes antes de actualizar los parámetros del modelo. Esto asegura que el modelo no diverja incluso cuando los gradientes aumentan en algún punto durante el proceso de entrenamiento.

Utilizamos la perplejidad para evaluar el modelo. Como se discutió en la Sección 8.4.4, esto asegura que las secuencias de diferente longitud sean comparables.

Específicamente, cuando se usa el particionamiento secuencial, inicializamos el estado oculto solo al comienzo de cada época. Desde el i
  El ejemplo de subsecuencia en el siguiente minilote es adyacente al i-ésimo actual
  ejemplo de subsecuencia, el estado oculto al final del minilote actual se usará para inicializar el estado oculto al comienzo del siguiente minilote. De esta forma, la información histórica de la secuencia almacenada en estado oculto podría fluir sobre subsecuencias adyacentes dentro de una época. Sin embargo, el cálculo del estado oculto en cualquier punto depende de todos los minilotes anteriores en la misma época, lo que complica el cálculo del gradiente. Para reducir el costo computacional, separamos el gradiente antes de procesar cualquier minilote para que el cálculo del gradiente del estado oculto siempre se limite a los pasos de tiempo en un minilote.

Cuando usamos el muestreo aleatorio, necesitamos reinicializar el estado oculto para cada iteración ya que cada ejemplo se muestrea con una posición aleatoria. Al igual que la función train_epoch_ch3 en la Sección 3.6, el actualizador es una función general para actualizar los parámetros del modelo. Puede ser la función d2l.sgd implementada desde cero o la función de optimización integrada en un marco de aprendizaje profundo.


In [2]:
    my ($net, $train_iter, $loss, $updater, $device, $use_random_iter) = @_;
    my ($state, $timer) = (undef, d2l::Timer->new());
    my $metric = d2l::Accumulator->new(2); # Sum of training loss, no. of tokens

    while (my ($X, $Y) = $train_iter->()) {
        if (!defined $state || $use_random_iter) {
            # Initialize `state` when either it is the first iteration or
            # using random sampling
            $state = $net->begin_state(batch_size => $X->shape->[0], ctx => $device);
        } else {
            $_->detach for @$state;
        }
        my $y = $Y->T->reshape(-1);
        $X = $X->as_in_ctx($device);
        $y = $y->as_in_ctx($device);

        autograd->record(sub {
            my ($y_hat, $state) = $net->($X, $state);
            my $l = $loss->($y_hat, $y)->mean;
            $l->backward;
        });

        grad_clipping($net, 1);
        $updater->(batch_size => 1); # Since the `mean` function has been invoked
        $metric->add($l * $y->size, $y->size);
    }
    return exp($metric->[0] / $metric->[1]), $metric->[1] / $timer->stop;
}

Error: Unmatched right curly bracket at reply input line 42, at end of line
syntax error at reply input line 42, near "}
}"



La función de entrenamiento admite un modelo RNN implementado desde cero o utilizando API de alto nivel.

In [3]:
#@save
sub train_epoch_ch8 {
    my ($net, $train_iter, $loss, $updater, $device, $use_random_iter) = @_;

    my ($state, $timer) = (undef, d2l::Timer->new());
    my $metric = d2l::Accumulator->new(2);  # Sum of training loss, no. of tokens

    while (my ($X, $Y) = $train_iter->()) {
        if (!defined $state || $use_random_iter) {
            # Initialize `state` when either it is the first iteration or
            # using random sampling
            $state = $net->begin_state(
                batch_size => $X->shape->[0],
                ctx => $device,
            );
        } else {
            $_->detach() for @$state;
        }

        my $y = $Y->T->reshape(-1);
        ($X, $y) = ($X->as_in_ctx($device), $y->as_in_ctx($device));

        autograd->with(autograd->record(), sub {
            my ($y_hat, $state) = $net->($X, $state);
            my $l = $loss->($y_hat, $y)->mean();
            $l->backward();
            grad_clipping($net, 1);
            $updater->(batch_size => 1);  # Since the `mean` function has been invoked
            $metric->add($l * $y->size, $y->size);
        });
    }

    return (
        exp($metric->[0] / $metric->[1]),
        $metric->[1] / $timer->stop(),
    );
}

sub train_ch8 {
    my ($net, $train_iter, $vocab, $lr, $num_epochs, $device,
        $use_random_iter) = @_;

    my $loss = gluon->loss->SoftmaxCrossEntropyLoss();
    my $animator = d2l::Animator->new(
        xlabel => 'epoch',
        ylabel => 'perplexity',
        legend => ['train'],
        xlim => [10, $num_epochs],
    );

    # Initialize
    if ($net->isa('gluon::Block')) {
        $net->initialize(
            ctx => $device,
            force_reinit => 1,
            init => init->Normal(0.01),
        );
        my $trainer = gluon->Trainer(
            $net->collect_params(),
            'sgd',
            {learning_rate => $lr},
        );
        my $updater = sub {
            my ($batch_size) = @_;
            $trainer->step($batch_size);
        };
    } else {
        my $updater = sub {
            my ($batch_size) = @_;
            d2l->sgd($net->params, $lr, $batch_size);
        };
    }

    my $predict = sub { predict_ch8($_[0], 50, $net, $vocab, $device) };

    # Train and predict
    for my $epoch (0 .. $num_epochs - 1) {
        my ($ppl, $speed) = train_epoch_ch8(
            $net, $train_iter, $loss, $updater, $device, $use_random_iter,
        );

        if (($epoch + 1) % 10 == 0) {
            $animator->add($epoch + 1, [$ppl]);
        }
    }

    print "perplexity $ppl, $speed tokens

Error: Can't find string terminator '"' anywhere before EOF at reply input line 87.



Ahora podemos entrenar el modelo RNN. Dado que solo usamos 10000 tokens en el conjunto de datos, el modelo necesita más épocas para converger mejor.


In [4]:
my $num_epochs = 500;
my $lr = 1;
train_ch8($net, $train_iter, $vocab, $lr, $num_epochs, d2l->try_gpu());


Error: Can't locate object method "try_gpu" via package "d2l" (perhaps you forgot to load "d2l"?) at reply input line 3.


Finalmente, verifiquemos los resultados del uso del método de muestreo aleatorio.


In [5]:
my $net = RNNModelScratch(scalar(@vocab), $num_hiddens, d2l->try_gpu(), $get_params, $init_rnn_state, $rnn);
train_ch8($net, $train_iter, \@vocab, $lr, $num_epochs, d2l->try_gpu(), 1);


Error: Can't locate object method "try_gpu" via package "d2l" (perhaps you forgot to load "d2l"?) at reply input line 1.


Aunque implementar el modelo RNN anterior desde cero es instructivo, no es conveniente. En la siguiente sección, veremos cómo mejorar el modelo RNN, por ejemplo, cómo hacerlo más fácil de implementar y hacer que se ejecute más rápido.

## 8.5.7. Resumen
* Podemos entrenar un modelo de lenguaje a nivel de caracteres basado en RNN para generar texto siguiendo el prefijo de texto proporcionado por el usuario.

* Un modelo de lenguaje RNN simple consta de codificación de entrada, modelado RNN y generación de salida.

* Los modelos RNN necesitan la inicialización del estado para el entrenamiento, aunque el muestreo aleatorio y la partición secuencial usan formas diferentes.

* Cuando usamos partición secuencial, necesitamos separar el gradiente para reducir el costo computacional.

* Un período de calentamiento permite que un modelo se actualice (p. ej., obtenga un mejor estado oculto que su valor inicializado) antes de realizar cualquier predicción.

* El recorte de degradado evita la explosión de degradado, pero no puede corregir los degradados que se desvanecen.




In [132]:
print $vocab->len;
print "\nTokens to ids: ", dump $vocab->getitem([ " ", "b", "u", "t", " ", "s", "o", "m", "e", " " ]);
print "\nIds to tokens: ", dump $vocab->to_tokens([ 1, 21, 14, 3, 1, 8, 6, 13, 2, 1 ]);

28
Tokens to ids: [1, 21, 14, 3, 1, 8, 6, 13, 2, 1]
Ids to tokens: [" ", "b", "u", "t", " ", "s", "o", "m", "e", " "]

1